In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [15]:
import boto3
import pandas as pd
import json
import re

<IPython.core.display.Javascript object>

In [3]:
client = boto3.client("cloudsearch")

Found credentials in shared credentials file: ~/.aws/credentials


<IPython.core.display.Javascript object>

In [4]:
domain_name = "mpn-mfg-2"

<IPython.core.display.Javascript object>

In [5]:
response = client.describe_domains(DomainNames=[domain_name])
doc_ep = response["DomainStatusList"][0]["DocService"]["Endpoint"]
search_ep = response["DomainStatusList"][0]["SearchService"]["Endpoint"]
print("doc_ep: ", doc_ep)
print("search_ep: ", search_ep)

doc_ep:  doc-mpn-mfg-2-35jf2boyu4jash2rk5vrcgpa34.us-west-2.cloudsearch.amazonaws.com
search_ep:  search-mpn-mfg-2-35jf2boyu4jash2rk5vrcgpa34.us-west-2.cloudsearch.amazonaws.com


<IPython.core.display.Javascript object>

In [6]:
doc_client = boto3.client("cloudsearchdomain", endpoint_url=f"https://{doc_ep}")

<IPython.core.display.Javascript object>

In [7]:
src_client = boto3.client("cloudsearchdomain", endpoint_url=f"https://{search_ep}")

<IPython.core.display.Javascript object>

In [8]:
df = pd.read_csv("ihs_stats_052022_v4.csv")
df.shape

(2303161, 13)

<IPython.core.display.Javascript object>

In [9]:
df.columns

Index(['mpn', 'manufacturer_ihs', 'manufacturer_support_ihs',
       'manufacturer_evidence_ihs', 'commodity_ihs', 'commodity_support_ihs',
       'commodity_evidence_ihs', 'sub_commodity_ihs',
       'sub_commodity_support_ihs', 'sub_commodity_evidence_ihs',
       'commodity_ld', 'sub_commodity_ld', 'manufacturer_ld'],
      dtype='object')

<IPython.core.display.Javascript object>

In [71]:
df["mpn"] = df["mpn"].str.replace(" ", "")
df["mpn"] = df["mpn"].str.replace("\\", "")

<IPython.core.display.Javascript object>

In [51]:
df.rename(columns={'support':'ihs_support', 'evidence':'ihs_evidence'}, inplace=True)

In [11]:
# df["manufacturer_ld"] = ""
# df["commodity_ld"] = ""
# df["sub_commodity_ld"] = ""
df["description_ihs"] = "ihs description"
df["date_ihs"] = "2022-05-01T00:00:00Z"

<IPython.core.display.Javascript object>

In [58]:
def find_nonalpha(text):
    # result = re.findall("[A-Za-z0-9 ]", text)
    # return result
    return "".join(filter(str.isalnum, text))


# df['nonalpha']=df['company_code'].apply(lambda x: find_nonalpha(x))

df["mpn_alphanum"] = df["mpn"].apply(lambda x: find_nonalpha(x))

<IPython.core.display.Javascript object>

In [23]:
df["manufacturer_evidence_ihs"] = df["manufacturer_evidence_ihs"].astype("int")
df["commodity_evidence_ihs"] = df["commodity_evidence_ihs"].astype("int")
df["sub_commodity_evidence_ihs"] = df["sub_commodity_evidence_ihs"].astype("int")

<IPython.core.display.Javascript object>

In [24]:
df.head()

,mpn,manufacturer_ihs,manufacturer_support_ihs,manufacturer_evidence_ihs,commodity_ihs,commodity_support_ihs,commodity_evidence_ihs,sub_commodity_ihs,sub_commodity_support_ihs,sub_commodity_evidence_ihs,commodity_ld,sub_commodity_ld,manufacturer_ld,description_ihs,date_ihs,mpn_alphanum
0,1PS76SB10,NEXPERIA,2,100,DIODES,4,100,RECTIFIER DIODES,4,100,DISCRETE_SEMICONDUCTOR_DEVICES,DIODES,NEXPERIA,ihs description,2022-05-01T00:00:00Z,1PS76SB10
1,B72207S0271K101,TDK,2,100,RESISTORS,4,100,NON-LINEAR RESISTORS,4,100,EE_PASSIVES,RESISTORS_NONLINEAR,TDK,ihs description,2022-05-01T00:00:00Z,B72207S0271K101
2,C0603X224K3RAC7867,KEMET,2,100,CAPACITORS,6,100,FIXED CAPACITORS,6,100,EE_PASSIVES,CAPACITORS_FIXED,KEMET,ihs description,2022-05-01T00:00:00Z,C0603X224K3RAC7867
3,CL05C101JB5NNNC,SEM,2,100,CAPACITORS,8,100,FIXED CAPACITORS,8,100,EE_PASSIVES,CAPACITORS_FIXED,SAMSUNG ELECTRO MECHANICS,ihs description,2022-05-01T00:00:00Z,CL05C101JB5NNNC
4,R82DC3470Z360J,KEMET,2,100,CAPACITORS,6,100,FIXED CAPACITORS,6,100,EE_PASSIVES,CAPACITORS_FIXED,KEMET,ihs description,2022-05-01T00:00:00Z,R82DC3470Z360J


<IPython.core.display.Javascript object>

In [48]:
df["manufacturer_evidence_ihs"].unique()

array([100,  66,  80,  50,  40,  75,  28,  85,  57,  60,  11])

<IPython.core.display.Javascript object>

In [50]:
df["manufacturer_evidence_ihs"].value_counts()

100    2300639
66        2449
50          27
80          12
60          11
40           9
57           4
75           3
28           3
85           2
11           2
Name: manufacturer_evidence_ihs, dtype: int64

<IPython.core.display.Javascript object>

In [52]:
df["commodity_evidence_ihs"].value_counts()

100    2303152
75           4
60           3
66           2
Name: commodity_evidence_ihs, dtype: int64

<IPython.core.display.Javascript object>

In [53]:
df["sub_commodity_evidence_ihs"].value_counts()

100    2303144
60           9
75           4
66           3
80           1
Name: sub_commodity_evidence_ihs, dtype: int64

<IPython.core.display.Javascript object>

In [27]:
data_dct = df.sample(10000).to_dict(orient="records")
data_dct[0]

{'mpn': 'RC0201J105-07-RG70',
 'manufacturer_ihs': 'TAITRON',
 'manufacturer_support_ihs': 2,
 'manufacturer_evidence_ihs': 100,
 'commodity_ihs': 'RESISTORS',
 'commodity_support_ihs': 2,
 'commodity_evidence_ihs': 100,
 'sub_commodity_ihs': 'FIXED RESISTORS',
 'sub_commodity_support_ihs': 2,
 'sub_commodity_evidence_ihs': 100,
 'commodity_ld': 'EE_PASSIVES',
 'sub_commodity_ld': 'FIXED RESISTORS',
 'manufacturer_ld': 'TAITRON COMPONENTS',
 'description_ihs': 'ihs description',
 'date_ihs': '2022-05-01T00:00:00Z',
 'mpn_alphanum': 'RC0201J10507RG70'}

<IPython.core.display.Javascript object>

In [82]:
data_dct = df[df["manufacturer_evidence_ihs"] < 99].to_dict(orient="records")
len(data_dct)

2522

<IPython.core.display.Javascript object>

In [97]:
mpns = ["NRC04J164TR50F", "CRCW120633K0JNECIF", "RCA040218R0FKEE", "NRC04J752HTR50F"]
mpns = ["RMCF1/16S3.92K1%"]
mpns = ["NRC04J5R6TR10F"]
dd = df[df["mpn"].isin(mpns)].to_dict(orient="records")
dd

[{'mpn': 'NRC04J5R6TR10F',
  'manufacturer_ihs': 'NIC',
  'manufacturer_support_ihs': 2,
  'manufacturer_evidence_ihs': 100,
  'commodity_ihs': 'RESISTORS',
  'commodity_support_ihs': 2,
  'commodity_evidence_ihs': 100,
  'sub_commodity_ihs': 'FIXED RESISTORS',
  'sub_commodity_support_ihs': 2,
  'sub_commodity_evidence_ihs': 100,
  'commodity_ld': 'EE_PASSIVES',
  'sub_commodity_ld': 'FIXED RESISTORS',
  'manufacturer_ld': 'NIC COMPONENTS',
  'description_ihs': 'ihs description',
  'date_ihs': '2022-05-01T00:00:00Z',
  'mpn_alphanum': 'NRC04J5R6TR10F'}]

<IPython.core.display.Javascript object>

In [84]:
data = []
for d in data_dct:
    data.append({"type": "add", "id": d["mpn"], "fields": d})
data[0]

{'type': 'add',
 'id': '04024D225MAT2A',
 'fields': {'mpn': '04024D225MAT2A',
  'manufacturer_ihs': 'KYOCERA AVX',
  'manufacturer_support_ihs': 3,
  'manufacturer_evidence_ihs': 66,
  'commodity_ihs': 'CAPACITORS',
  'commodity_support_ihs': 3,
  'commodity_evidence_ihs': 100,
  'sub_commodity_ihs': 'FIXED CAPACITORS',
  'sub_commodity_support_ihs': 3,
  'sub_commodity_evidence_ihs': 100,
  'commodity_ld': 'EE_PASSIVES',
  'sub_commodity_ld': 'CAPACITORS_FIXED',
  'manufacturer_ld': 'AVX CORPORATION',
  'description_ihs': 'ihs description',
  'date_ihs': '2022-05-01T00:00:00Z',
  'mpn_alphanum': '04024D225MAT2A'}}

<IPython.core.display.Javascript object>

In [98]:
data = []
for d in dd:
    data.append({"type": "add", "id": d["mpn"], "fields": d})
data[0]

{'type': 'add',
 'id': 'NRC04J5R6TR10F',
 'fields': {'mpn': 'NRC04J5R6TR10F',
  'manufacturer_ihs': 'NIC',
  'manufacturer_support_ihs': 2,
  'manufacturer_evidence_ihs': 100,
  'commodity_ihs': 'RESISTORS',
  'commodity_support_ihs': 2,
  'commodity_evidence_ihs': 100,
  'sub_commodity_ihs': 'FIXED RESISTORS',
  'sub_commodity_support_ihs': 2,
  'sub_commodity_evidence_ihs': 100,
  'commodity_ld': 'EE_PASSIVES',
  'sub_commodity_ld': 'FIXED RESISTORS',
  'manufacturer_ld': 'NIC COMPONENTS',
  'description_ihs': 'ihs description',
  'date_ihs': '2022-05-01T00:00:00Z',
  'mpn_alphanum': 'NRC04J5R6TR10F'}}

<IPython.core.display.Javascript object>

In [99]:
data = json.dumps(data)
# data

<IPython.core.display.Javascript object>

In [100]:
response = src_client.upload_documents(documents=data, contentType="application/json")
print(response["ResponseMetadata"]["HTTPStatusCode"])

200


<IPython.core.display.Javascript object>

In [66]:
data

'[{"type": "add", "id": "RMCF1/16S3.92K1%", "fields": {"mpn": "RMCF1/16S3.92K1%", "manufacturer_ihs": "SEI ELEK", "manufacturer_support_ihs": 2, "manufacturer_evidence_ihs": 100, "commodity_ihs": "RESISTORS", "commodity_support_ihs": 8, "commodity_evidence_ihs": 100, "sub_commodity_ihs": "FIXED RESISTORS", "sub_commodity_support_ihs": 8, "sub_commodity_evidence_ihs": 100, "commodity_ld": "EE_PASSIVES", "sub_commodity_ld": "FIXED RESISTORS", "manufacturer_ld": "STACKPOLE ELECTRONICS", "description_ihs": "ihs description", "date_ihs": "2022-05-01T00:00:00Z", "mpn_alphanum": "RMCF116S392K1"}}]'

<IPython.core.display.Javascript object>

In [132]:
for d in data_dct:
    print(d["mpn"])

NRC04J164TR50F
CRCW120633K0JNECIF
RCA040218R0FKEE
NTRH06B3900CTRF
BZT52C20-HE3-18
RM04JTN822
WK04K39R2FTL
C0603X561K5RACAUTO
SN74HCS125DR
T493C157M006CH62207640
501S43W473KV6E-RC
VJ0402A560FNAAP
0603WGF7501TDE
C1210T473J5GBG7210
HM2P71PDE121GEN
T520V476M016ASE0457280
SG3040LC
RM12J300CTG
C0805T510G5GBGT500
USBLC62SC6
GMC31X7R106K16NTD
P0603E4071BG
BCTF000201-1300FTH
RMCF1/16S3.92K1%
HCHP0603K3571FNPD
CPR1206O26R7FE
C0402T271G5GBCTU
06035A120G4T2A
AS05FBE1651
K7R321882C-EC250
MJD44H11T4-A
CHP0603K7502JWTD
NRC-AS04F6041TRQYF
CR0402-FX-9091GLF
RL12F6981CTD
ACTA000805-53R6FTP
RC2012F2872AS
GMC04CG910J50PTD
CR-06FPA--174R
CRCW0603698KFKEI
BCSR000805B1823FTP
ERJ1GNJ123C
WF06H1051DTL
T493A155K035CH61107640
RK73G1JLBK6802D
TAZE475K015CWLB5745
C0603T153F3GCC7411
AU06ZC106KAG2A
AA0402FR-07182RL
WF12K1240FGL
0805HTR50TGLC
RC02W124JT
MC0805-3742-FT
CR1/81780FV
BCTS000603B1400FTP
500R07S2R2CG4Y
0-5161390-2
NRCA25F2493TR10QYF
GMT31X7R473J50NT-15
RK73H1JTBK1334F
MC1206B-100-JTW
SR151E103MARTR1
CR-0AF

PWR05FTEW2802N
WR08X75R0FTL
GGM1885C2A300GA16J
SAS21-30R0-FY-A
RK73G2ALTDD7681F
1206B103K500NB
ERJ-8ENF-12R0V
0805B332K201CT
T545W476M016ATE0457280
BCTR000805-9531FTP
C1206X392G5GAC7210
0805B273J500CG
WR12X153JQR
T493C336M016CC62207640
MR12W1R0JGL
CRCW060336K5FKEB
AS03FVE40R2A
AS10FTF1402
TPSC475K050R0800
71321LA55JGI
VJ1812A221JNHAT
239044742103L
XC7A35T-L2CSG324E
WR06X12R1FGR
NRC04F4423HTR50F
D11/CRCW06031005M61%ET4E3
RE1V476M08005PA
GS8662S09BD-400T
C1206X105K4RAL7215
VJ1206A3R3CFEAI2M
CRCW0201620KFNEE
1471MAM1625RXN1110
MRS25000C3300FC100
TC74LCX138FT(EL)
C346C474M1U5TATR
TNPW12065K00BEEA
N3372-53K2UB
P0603H1473FGTB
T493C107K006BK62107640
500R14N101JT4E
RN06B2151CT5-X25
CR0402-FX-2802GLF
CDF0402619K1%100PPM/KNP5
T520D337M6R3AME0407280
TA020TCM156MDR-13
NRC-AS10PF1270TRF
C0603T821K5RAC7411
SF04R7410ATLA
D55342K07B100DPT1M
0603F0505P60DCBS02A
RCA04021M30FKEE
250R14W102JV4T
GWCR0201-910RFT15
RMC16-155FB
RCA08051K85%200ET1E3
AS06FWE1501
100B3R0BTN500X
842-816-3439-619
0603HC-3N6XJSY
23

RCA0603680RJNEDIF
SAS05-121-JY-B
0402RN270J201KT
C1206C220F5GACAUTO7411
RPGW16K2000FB
M55342K08D51G0P
TL1105EF100Q
ORE561M0GBU-0808KS
CEC2F47PF1%100V
SIW3000GIP1-T13
RC0201FR-07187KP
0402N1R2B101D
TPS2032QDRQ1
RBS06ATP3401
UMB02070E1263AB1
C0603T609C5GCL7411
T493C106K020AT61107610
WR08W1784FBL
13670732
C0603C-5N10G1T4
ASAS10-63R4-FY
C1206V473KCRAC7215
CR0402-16W-56R2FPT-13
WR04X27R4FAL
TBJD157K006RKLC0945
CR-01FLA--2K21
R523R4470BBP0K
M55342K09U1T00M
P6SMB51C_R1_00001
HCHP0603K1584FBT131
RK73G2ALTP3483F
APVH03W11K5BET
NRC-AS10J685TRQYF
251R15W332KV6U
CL05C2R4AB5NNNL
RK73H2ARTTP2872F
C0805C629C5GAL3325
RL04F4421CT
MPMT20018001AT0
CDF-S0402402R1%100PPM/KNB10
RLR05C1052FM
RM12F5761CTG-X
C0603X7R102KFT
SR0805FR-078K2L
RK73B2BLTC226J
HCHP0603K1742FWT131
251SHS390FA
SR0402FR-7W28KL
T493D107M010CC611C7280
DS1340Z-33/T&R
C1206C104KARAL7210
CR16-5102-FL
B12J3R0E
CDI040282K5%100PPM/KNP10
TBJC226M016JSLC90OR
PFC-W0805ASLF-12-5600-B
CL10C510JB8GNO
AD8228BRZ
D2F-01FL3-D3
CHP0603K1300JNTC
CE0402-331

VJ0402Y681KXAAC2MP
T493C336K010AN63207280
C1812T474K1RBLTM
LQG15HN10NG02B
NIS5132MN3TXG
CC0805JPNPO9BN152
VJ0603A0R5BXATW1BC
TBJB686M010LRSZ0H24
RK73B1HTTC2R00J
T491A685K010AN
RCMS0211802FHS14
T493D226M016CC63107640
239064741692L
EMVZ160ADA101MF60G
C1206T182J5GCC
C1206X274K3RAC3325
CR0603AFX-8253ELF
GMC31CG103F50NTG
RNR50J1022BMRSL31
T493C157M006CN64107280
AS01FWF38873
TBJB106M010RSLC90OR
RK73B1JGTDD514J
HH15N120J500CG
NRCL12J223TR20F
CE1210-226-K006XTW
WSL2010R0600FEK
MAX3222EETP+T
C0805T153J1GBL7210
TL431BSA7
MBB0207VD3013BRP00
VJ0603A100FEBAI68
MC0402-1472-FB101W
RC0603FR-13953KL
RK73B2AXTD511J
C0603T302J5GBL7081
P0603E2153BBTC
CWR11JK156MC
CDF-N0402732K1%100PPM/KNP10
RMH04FT2R7
W3A41A221KAT2A
12065C563MAT2A
C0805Y123K5RAL
CDF-K040217K41%100PPM/KNP10
D2512068B1781FP0
WR08X5113FGLJ
SMM0204VE1300BB300
P0603E3831WBPA
VJ0805A681JXAAT5G
9C12063A9R09FLAFT
C0402T151K3GBGTU
CPW10-35R7-FK
CNC12YGF100NF10%200VBAT6
MR06W4754FTL
CMF5527K400FKBF70
RCG06032002K45%EB
0603N120F500DHW-HB
RAC02W3402F

952926CGLFT
HCHP0603K45R3FBPC
0402CG100J500ST
1101MBMAB25RXN711P
WCR0805LF-2553-F-P-LT
CR16-431-JL
MF1/4DCT52A5903F
T493A225K020CH62207640
008263043200001
WR08W1784FGL
RN06A3322CT5X15
CR-05FE6--3K16
RNCF0603AKE13K5
CMF552K1500BHBF70
GRP1555C1H100RZ01E
AD9516-4BCPZ
RN06A3012CT525
NMC0402X7R102M25TRP
SY006M0150A5F-0511
C1210T105J5RCLTU
RC02W5761FBL
CDF040218R21%100PPM/KNP10
RK73H1JRTTD5101F
C0402T183K4RAG7411
MLK0603L18NJT000
IR3883MTRPBF
EKZE101ETD221MK25S
RC0603F135AS
ERJ1GEF18R7C
RT9198-33PBR
CRCW06031K50JNTA
RS-06K8061FT
ACTA000805-243XJTP
RCA0402107RFKED
CRCW0201270RFNED
CR-03FL6--1K65
9C02017A1100JLPF7
AR05BBCV1372
LM49350RL
RM10F43R2CTD-U
AW-08-03-G-D-150-155-P
T493C155K035AB6320
AR16-1372-FK
T543Y687M004AHE005
281-324
RT1206FRD0780K6L
AS05JTE0390
VJ0805A221GNAAP5G
RM06J561CTD-X
RK73H1JLTP7150F
219-4ESJ
NRC06F9091TR10F
D12-CRCW08051004K421%P0
251R15S220JC3U
WSL25126L500FBA
CMF-551743FT-9R36
CL05B103KO5GNND
CEC14YCWF150PF2%50VS8
100A1R8CP150XT500
0805W8J0103T1E
EKY-100ELL681MJC5S
B

<IPython.core.display.Javascript object>

In [26]:
df[["mpn", "mpn_alphanum"]].sample(50)

,mpn,mpn_alphanum
256747,8152LDMAB25RXN4110,8152LDMAB25RXN4110
1004045,CRCW0805681KFKTC,CRCW0805681KFKTC
386569,AS03FWE9090A,AS03FWE9090A
413820,ASAS16-1621-FL,ASAS161621FL
2052623,T493C226K010BK61107640,T493C226K010BK61107640
...,...,...
509549,C0402C472K9RAC,C0402C472K9RAC
2007064,SR0805FR-7T107KL,SR0805FR7T107KL
1102067,ERG2SJ683,ERG2SJ683
900599,CR-05FCA--680K,CR05FCA680K


<IPython.core.display.Javascript object>

In [93]:
dft = pd.read_csv("part_052022_v2.csv")
dft[dft["Imported_Mfr_PN"] == "BC81740"].head(20)

/Users/sridharkannam/opt/anaconda3/envs/py11/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3172: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Imported_Mfr_PN,Imported_Mfr,Part_Type,Part_Category
241033,BC81740,CENTRAL SEMI,Transistors,Small Signal Bipolar Transistors
241036,BC81740,CDIL,Transistors,Small Signal Bipolar Transistors
241039,BC81740,CENTRAL SEMI,Transistors,Small Signal Bipolar Transistors
241042,BC81740,MCC,Transistors,Small Signal Bipolar Transistors
241045,BC81740,NEXPERIA,Transistors,Small Signal Bipolar Transistors
...,...,...,...,...
3505560,BC81740,CENTRAL SEMI,Transistors,Small Signal Bipolar Transistors
3505563,BC81740,MCC,Transistors,Small Signal Bipolar Transistors
3505566,BC81740,NEXPERIA,Transistors,Small Signal Bipolar Transistors
3505569,BC81740,PANJIT,Transistors,Small Signal Bipolar Transistors


<IPython.core.display.Javascript object>

In [96]:
dft[dft["Imported_Mfr_PN"] == "BC81740"]["Imported_Mfr"].value_counts()

CENTRAL SEMI    4
MCC             2
CDIL            2
ZETEX           2
PANJIT          2
NEXPERIA        2
Name: Imported_Mfr, dtype: int64

<IPython.core.display.Javascript object>